In [14]:
import re
import numpy as np
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError, GeocoderUnavailable
from retrying import retry
import time
import folium

In [15]:
df = pd.read_json('./jobscraper/jobscraper/it_jobs.json')
df

,title,city,url,details
0,Викладач програмування (нічна зміна),Дистанційно,https://www.work.ua/jobs/5708935/,"{'salary': ['9000', '19500 '], 'company': 'Jus..."
1,Фахівець текстової підтримки,"Київ,",https://www.work.ua/jobs/5668550/,"{'salary': ['13000', '18000 '], 'company': 'Me..."
2,Менеджер з продажу електроніки (7 км) в інтерн...,"Одеса,",https://www.work.ua/jobs/5043547/,"{'salary': ['25000', '28000 '], 'company': 'Ав..."
3,Microsoft CRM Developer,Київ,https://www.work.ua/jobs/5613679/,"{'salary': None, 'company': 'Агропросперіс', '..."
4,Продавець-консультант техніки Apple,"Київ,",https://www.work.ua/jobs/4850818/,"{'salary': ['23000', '46000 '], 'company': 'IP..."
...,...,...,...,...
5898,Головний спеціаліст з інформаційних технологій,"Львів,",https://www.work.ua/jobs/5638578/,"{'salary': ['9880 '], 'company': 'Галицький ра..."
5899,SMM-менеджер,"Київ,",https://www.work.ua/jobs/5318458/,"{'salary': ['25000 '], 'company': 'Бережанськи..."
5900,Адміністратор соц. мереж,Київ,https://www.work.ua/jobs/5624384/,"{'salary': ['23000', '25000 '], 'company': 'De..."
5901,Game Designer,Дистанційно,https://www.work.ua/jobs/3360940/,"{'salary': None, 'company': 'Alta Games', 'com..."


In [16]:
df1 = df[['title', 'city', 'url']]
df1

,title,city,url
0,Викладач програмування (нічна зміна),Дистанційно,https://www.work.ua/jobs/5708935/
1,Фахівець текстової підтримки,"Київ,",https://www.work.ua/jobs/5668550/
2,Менеджер з продажу електроніки (7 км) в інтерн...,"Одеса,",https://www.work.ua/jobs/5043547/
3,Microsoft CRM Developer,Київ,https://www.work.ua/jobs/5613679/
4,Продавець-консультант техніки Apple,"Київ,",https://www.work.ua/jobs/4850818/
...,...,...,...
5898,Головний спеціаліст з інформаційних технологій,"Львів,",https://www.work.ua/jobs/5638578/
5899,SMM-менеджер,"Київ,",https://www.work.ua/jobs/5318458/
5900,Адміністратор соц. мереж,Київ,https://www.work.ua/jobs/5624384/
5901,Game Designer,Дистанційно,https://www.work.ua/jobs/3360940/


In [17]:
df2 = df['details']
df2 = df2.apply(lambda x: pd.Series(x))
df2

,salary,company,company_link,address,job_type,tags,description
0,"[9000, 19500 ]",JustSchool,https://www.work.ua/jobs/by-company/1455210/,None,"Повна зайнятість, неповна зайнятість. Також го...","[Python, Здатність до навчання, Програмування,...",
1,"[13000, 18000 ]",Megogo,https://www.work.ua/jobs/by-company/240655/,"Київ, вулиця Рибальська, 22.",Повна зайнятість. Також готові взяти студента.,"[Стресостійкість, Користувач ПК, Грамотна усна...","MEGOGO — медіасервіс для тих, хто цінує свобод..."
2,"[25000, 28000 ]","Авеон, інтернет-магазин",https://www.work.ua/jobs/by-company/2487995/,"Одеса, Овідіопольська дорога, 7.",Повна зайнятість.,"[Ведение CRM, Прием и обработка заказов]","Ми — інтернет-магазин побутових товарів, елект..."
3,None,Агропросперіс,https://www.work.ua/jobs/by-company/125534/,Київ,Повна зайнятість. Досвід роботи від 1 року.,[],
4,"[23000, 46000 ]",IPeople,https://www.work.ua/jobs/by-company/523846/,"Київ, вулиця Берковецька, 6Д.",Повна зайнятість. Досвід роботи від 1 року.,"[Комунікабельність, Впевненість, Продаж, Робот...","iPeople — це Компанія, яка обʼєднує Інновації ..."
...,...,...,...,...,...,...,...
5898,[9880 ],Галицький районний суд м. Львова,https://www.work.ua/jobs/by-company/2669586/,"Львів, вулиця Чоловського, 2.",Повна зайнятість. Також готові взяти студента....,"[MS Office, Ремонт ПК, Користувач ПК, Відповід...",Вимоги : вища освіта ступеня молодшого бакала...
5899,[25000 ],"Бережанський В.С., ФОП",https://www.work.ua/jobs/by-company/2558697/,"Київ, Велика Васильківська, 137.",Повна зайнятість. Також готові взяти студента.,"[Відповідальність, SMM, Фотозйомка, Робота в к...",Ми шукаємо творчого та відповідального менедже...
5900,"[23000, 25000 ]",Decor Interiors,https://www.work.ua/jobs/by-company/2662501/,Київ,Повна зайнятість. Також готові взяти студента.,"[Adobe Photoshop, Canva, Знання графічних реда...",Ми займаємось роздрібною торгівлею предметів д...
5901,None,Alta Games,https://www.work.ua/jobs/by-company/1434329/,None,Повна зайнятість. Досвід роботи від 2 років.,"[Unity, Ігровий дизайн, Креативність]",Ми шукаємо досвідченого Game Designer для ство...


In [18]:
df = df1.merge(df2, how='left', on=df1.index)
df.drop(columns=['key_0'], inplace=True)

In [19]:
df.head()

,title,city,url,salary,company,company_link,address,job_type,tags,description
0,Викладач програмування (нічна зміна),Дистанційно,https://www.work.ua/jobs/5708935/,"[9000, 19500 ]",JustSchool,https://www.work.ua/jobs/by-company/1455210/,None,"Повна зайнятість, неповна зайнятість. Також го...","[Python, Здатність до навчання, Програмування,...",
1,Фахівець текстової підтримки,"Київ,",https://www.work.ua/jobs/5668550/,"[13000, 18000 ]",Megogo,https://www.work.ua/jobs/by-company/240655/,"Київ, вулиця Рибальська, 22.",Повна зайнятість. Також готові взяти студента.,"[Стресостійкість, Користувач ПК, Грамотна усна...","MEGOGO — медіасервіс для тих, хто цінує свобод..."
2,Менеджер з продажу електроніки (7 км) в інтерн...,"Одеса,",https://www.work.ua/jobs/5043547/,"[25000, 28000 ]","Авеон, інтернет-магазин",https://www.work.ua/jobs/by-company/2487995/,"Одеса, Овідіопольська дорога, 7.",Повна зайнятість.,"[Ведение CRM, Прием и обработка заказов]","Ми — інтернет-магазин побутових товарів, елект..."
3,Microsoft CRM Developer,Київ,https://www.work.ua/jobs/5613679/,None,Агропросперіс,https://www.work.ua/jobs/by-company/125534/,Київ,Повна зайнятість. Досвід роботи від 1 року.,[],
4,Продавець-консультант техніки Apple,"Київ,",https://www.work.ua/jobs/4850818/,"[23000, 46000 ]",IPeople,https://www.work.ua/jobs/by-company/523846/,"Київ, вулиця Берковецька, 6Д.",Повна зайнятість. Досвід роботи від 1 року.,"[Комунікабельність, Впевненість, Продаж, Робот...","iPeople — це Компанія, яка обʼєднує Інновації ..."


In [20]:
def count_salaryI(salary_list):
    try:
        salary_list = list(map(lambda x: int(x), salary_list))
    except:
        return np.nan
    if salary_list is None:
        return np.nan
    elif (num_ele:=len(salary_list)) > 1:
        return sum(salary_list)/num_ele
    else: return salary_list[0]

In [21]:
office_df = df[~df['city'].str.contains('Дистанційно', case=False)]
office_df['address'] = office_df['address'].apply(lambda x: np.nan if x is None else x)
office_df['salary'] = office_df['salary'].apply(count_salaryI) 
office_df = office_df.dropna()
office_df.head()

C:\Users\sit4e\AppData\Local\Temp\ipykernel_8128\3857246917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  office_df['address'] = office_df['address'].apply(lambda x: np.nan if x is None else x)
C:\Users\sit4e\AppData\Local\Temp\ipykernel_8128\3857246917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  office_df['salary'] = office_df['salary'].apply(count_salaryI)


,title,city,url,salary,company,company_link,address,job_type,tags,description
1,Фахівець текстової підтримки,"Київ,",https://www.work.ua/jobs/5668550/,15500.0,Megogo,https://www.work.ua/jobs/by-company/240655/,"Київ, вулиця Рибальська, 22.",Повна зайнятість. Також готові взяти студента.,"[Стресостійкість, Користувач ПК, Грамотна усна...","MEGOGO — медіасервіс для тих, хто цінує свобод..."
2,Менеджер з продажу електроніки (7 км) в інтерн...,"Одеса,",https://www.work.ua/jobs/5043547/,26500.0,"Авеон, інтернет-магазин",https://www.work.ua/jobs/by-company/2487995/,"Одеса, Овідіопольська дорога, 7.",Повна зайнятість.,"[Ведение CRM, Прием и обработка заказов]","Ми — інтернет-магазин побутових товарів, елект..."
4,Продавець-консультант техніки Apple,"Київ,",https://www.work.ua/jobs/4850818/,34500.0,IPeople,https://www.work.ua/jobs/by-company/523846/,"Київ, вулиця Берковецька, 6Д.",Повна зайнятість. Досвід роботи від 1 року.,"[Комунікабельність, Впевненість, Продаж, Робот...","iPeople — це Компанія, яка обʼєднує Інновації ..."
7,Администратор интернет-магазина,Дніпро,https://www.work.ua/jobs/5684272/,19500.0,"Сизонова Н.Г., ФОП",https://www.work.ua/jobs/by-company/2691728/,Дніпро,Повна зайнятість. Також готові взяти студента.,"[MS Office, Коммуникабельность, Ответственност...","Вакансія: Адміністратор інтернет-магазина Ми, ..."
11,"Фахівець з налаштування техніки (ТРЦ ""Республі...","Київ,",https://www.work.ua/jobs/5447098/,30000.0,Фокстрот,https://www.work.ua/jobs/by-company/9233/,"Київ, Кільцева дорога, 1.",Повна зайнятість.,[],Запрошуємо кандидатів з військовим та бойовим ...


In [22]:
clean_office_df = office_df[~(office_df['city'] == office_df['address'])]
clean_office_df

,title,city,url,salary,company,company_link,address,job_type,tags,description
1,Фахівець текстової підтримки,"Київ,",https://www.work.ua/jobs/5668550/,15500.0,Megogo,https://www.work.ua/jobs/by-company/240655/,"Київ, вулиця Рибальська, 22.",Повна зайнятість. Також готові взяти студента.,"[Стресостійкість, Користувач ПК, Грамотна усна...","MEGOGO — медіасервіс для тих, хто цінує свобод..."
2,Менеджер з продажу електроніки (7 км) в інтерн...,"Одеса,",https://www.work.ua/jobs/5043547/,26500.0,"Авеон, інтернет-магазин",https://www.work.ua/jobs/by-company/2487995/,"Одеса, Овідіопольська дорога, 7.",Повна зайнятість.,"[Ведение CRM, Прием и обработка заказов]","Ми — інтернет-магазин побутових товарів, елект..."
4,Продавець-консультант техніки Apple,"Київ,",https://www.work.ua/jobs/4850818/,34500.0,IPeople,https://www.work.ua/jobs/by-company/523846/,"Київ, вулиця Берковецька, 6Д.",Повна зайнятість. Досвід роботи від 1 року.,"[Комунікабельність, Впевненість, Продаж, Робот...","iPeople — це Компанія, яка обʼєднує Інновації ..."
11,"Фахівець з налаштування техніки (ТРЦ ""Республі...","Київ,",https://www.work.ua/jobs/5447098/,30000.0,Фокстрот,https://www.work.ua/jobs/by-company/9233/,"Київ, Кільцева дорога, 1.",Повна зайнятість.,[],Запрошуємо кандидатів з військовим та бойовим ...
12,Адміністратор інтернет-магазину,"Київ,",https://www.work.ua/jobs/5730429/,18500.0,StockHouse,https://www.work.ua/jobs/by-company/961447/,"Київ, вулиця Сирецька, 31.",Повна зайнятість. Також готові взяти людину з ...,"[MS Office, Комунікабельність, Відповідальніст...","Ми, компанія «StockHouse», шукаємо співробітни..."
...,...,...,...,...,...,...,...,...,...,...
5881,Системний адміністратор (с. Ходорівці),"Кам'янець-Подільський,",https://www.work.ua/jobs/5145553/,19000.0,"Авант-Агро, ПП",https://www.work.ua/jobs/by-company/975931/,"Кам'янець-Подільський, вулиця Шевченка, 2.",Повна зайнятість. Досвід роботи від 1 року.,"[Користувач ПК, Управління персоналом, Системн...",Приватне підприємство «Авант-Агро» є об`єктом ...
5883,Інтернет-маркетолог,"Львів,",https://www.work.ua/jobs/5690486/,35000.0,"Автопоміч, ТОВ",https://www.work.ua/jobs/by-company/1915939/,"Львів, вулиця Академіка Андрія Сахарова, 33.",Повна зайнятість. Досвід роботи від 1 року. Ви...,"[Просування в Instagram, Facebook Ads Manager,...",Компанія «Автопоміч» — всеукраїнський центр ад...
5898,Головний спеціаліст з інформаційних технологій,"Львів,",https://www.work.ua/jobs/5638578/,9880.0,Галицький районний суд м. Львова,https://www.work.ua/jobs/by-company/2669586/,"Львів, вулиця Чоловського, 2.",Повна зайнятість. Також готові взяти студента....,"[MS Office, Ремонт ПК, Користувач ПК, Відповід...",Вимоги : вища освіта ступеня молодшого бакала...
5899,SMM-менеджер,"Київ,",https://www.work.ua/jobs/5318458/,25000.0,"Бережанський В.С., ФОП",https://www.work.ua/jobs/by-company/2558697/,"Київ, Велика Васильківська, 137.",Повна зайнятість. Також готові взяти студента.,"[Відповідальність, SMM, Фотозйомка, Робота в к...",Ми шукаємо творчого та відповідального менедже...


In [23]:
df_for_map = clean_office_df[['title', 'address', 'description']]
df_for_map

,title,address,description
1,Фахівець текстової підтримки,"Київ, вулиця Рибальська, 22.","MEGOGO — медіасервіс для тих, хто цінує свобод..."
2,Менеджер з продажу електроніки (7 км) в інтерн...,"Одеса, Овідіопольська дорога, 7.","Ми — інтернет-магазин побутових товарів, елект..."
4,Продавець-консультант техніки Apple,"Київ, вулиця Берковецька, 6Д.","iPeople — це Компанія, яка обʼєднує Інновації ..."
11,"Фахівець з налаштування техніки (ТРЦ ""Республі...","Київ, Кільцева дорога, 1.",Запрошуємо кандидатів з військовим та бойовим ...
12,Адміністратор інтернет-магазину,"Київ, вулиця Сирецька, 31.","Ми, компанія «StockHouse», шукаємо співробітни..."
...,...,...,...
5881,Системний адміністратор (с. Ходорівці),"Кам'янець-Подільський, вулиця Шевченка, 2.",Приватне підприємство «Авант-Агро» є об`єктом ...
5883,Інтернет-маркетолог,"Львів, вулиця Академіка Андрія Сахарова, 33.",Компанія «Автопоміч» — всеукраїнський центр ад...
5898,Головний спеціаліст з інформаційних технологій,"Львів, вулиця Чоловського, 2.",Вимоги : вища освіта ступеня молодшого бакала...
5899,SMM-менеджер,"Київ, Велика Васильківська, 137.",Ми шукаємо творчого та відповідального менедже...


In [27]:
@retry(stop_max_attempt_number=10, wait_fixed=2000)
def geocode_with_retry(address):
    global geolocator
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return (location.latitude, location.longitude)
        else:
            return None
    except (GeocoderTimedOut, GeocoderServiceError, GeocoderUnavailable) as e:
        print(f"Error geocoding {address}: {e}")
        raise
    except:
        print("Unknown Error")
        raise

In [28]:
geolocator = Nominatim(user_agent="myGeocoder")

In [29]:
df_for_map['lat_long'] = df_for_map['address'].apply(geocode_with_retry)

Error geocoding Київ, вулиця Рибальська, 22.: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%9A%D0%B8%D1%97%D0%B2%2C+%D0%B2%D1%83%D0%BB%D0%B8%D1%86%D1%8F+%D0%A0%D0%B8%D0%B1%D0%B0%D0%BB%D1%8C%D1%81%D1%8C%D0%BA%D0%B0%2C+22.&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B904837D70>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=10)'))


C:\Users\sit4e\AppData\Local\Temp\ipykernel_8128\3204300567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_map['lat_long'] = df_for_map['address'].apply(geocode_with_retry)


In [30]:
df_for_map

,title,address,description,lat_long
1,Фахівець текстової підтримки,"Київ, вулиця Рибальська, 22.","MEGOGO — медіасервіс для тих, хто цінує свобод...","(50.4361756, 30.536542)"
2,Менеджер з продажу електроніки (7 км) в інтерн...,"Одеса, Овідіопольська дорога, 7.","Ми — інтернет-магазин побутових товарів, елект...","(46.44939755, 30.66345093166207)"
4,Продавець-консультант техніки Apple,"Київ, вулиця Берковецька, 6Д.","iPeople — це Компанія, яка обʼєднує Інновації ...","(50.4956041, 30.3593435)"
11,"Фахівець з налаштування техніки (ТРЦ ""Республі...","Київ, Кільцева дорога, 1.",Запрошуємо кандидатів з військовим та бойовим ...,"(50.374746, 30.45134252663972)"
12,Адміністратор інтернет-магазину,"Київ, вулиця Сирецька, 31.","Ми, компанія «StockHouse», шукаємо співробітни...","(50.49032255, 30.450095678238036)"
...,...,...,...,...
5881,Системний адміністратор (с. Ходорівці),"Кам'янець-Подільський, вулиця Шевченка, 2.",Приватне підприємство «Авант-Агро» є об`єктом ...,"(48.6891965, 26.58007834009785)"
5883,Інтернет-маркетолог,"Львів, вулиця Академіка Андрія Сахарова, 33.",Компанія «Автопоміч» — всеукраїнський центр ад...,"(49.82233185, 24.01674057847385)"
5898,Головний спеціаліст з інформаційних технологій,"Львів, вулиця Чоловського, 2.",Вимоги : вища освіта ступеня молодшого бакала...,"(49.841286800000006, 24.0016994585851)"
5899,SMM-менеджер,"Київ, Велика Васильківська, 137.",Ми шукаємо творчого та відповідального менедже...,"(50.4385776, 30.5159087)"


In [31]:
df_for_map.dropna(inplace=True)

C:\Users\sit4e\AppData\Local\Temp\ipykernel_8128\791492166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_map.dropna(inplace=True)


In [32]:
LATITUDE = 48.3794
LONGITUDE = 31.1656
min_lon, max_lon = 22, 41
min_lat, max_lat = 44, 53
job_map = folium.Map(location=[LATITUDE, LONGITUDE], control_scale=True, zoom_start=6.4, min_lat=min_lat, max_lat=max_lat, min_lon=min_lon, max_lon=max_lon, max_bounds=True, zoom_control=False,)
folium.CircleMarker([max_lat, min_lon], tooltip="Upper Left Corner").add_to(job_map)
folium.CircleMarker([min_lat, min_lon], tooltip="Lower Left Corner").add_to(job_map)
folium.CircleMarker([min_lat, max_lon], tooltip="Lower Right Corner").add_to(job_map)
folium.CircleMarker([max_lat, max_lon], tooltip="Upper Right Corner").add_to(job_map)

In [33]:
for name, address, desc, loc in df_for_map.values:
    html = f'<h1>{name}</h1>'
    folium.Marker(location=loc, popup=html, icon=folium.Icon(color='red')).add_to(job_map)

In [34]:
job_map

In [35]:
print('end')

end
